In [ ]:
import os
import sys
import time
import json
import shutil
import codecs
import requests
import psycopg2
import feedparser
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

sys.path.append("../")

import utils.NYTParser
from utils.sql_utils import *
from utils.scrape_utils import *
from utils.data_science_utils import *

In [ ]:
pipelineNYT(name='nyt_20200723_1600', loadDisk=True)
pipelineNYT(name='nyt_20200722_1000', loadDisk=True)
pipelineNYT(name='nyt_20200727_1200', loadDisk=True)

In [ ]:
pipelineNYTHeadlines(fetchSource=True, verbose=Verbose.INFO)

In [ ]:
pipelineNYTNewsWire(fetchSource=True, verbose=Verbose.INFO) # startPage = 0

In [ ]:
x = getRawArticles()

In [ ]:
x.shape

In [ ]:
pipelineAPINewsHeadline(fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
missing_sources = []
for topic in ['people', 'fashion', 'politics', 'COVID', 'elections', 'sport', 'news', 'war', 'world', 'europe', 'technology', 'science', 'movie', 'Trump', 'Biden', 'news', 'business', 'trade', 'stock', 'S&P500', 'tax', 'jobs', 
'trends', 'design', 'tv']:
    m_s = pipelineAPINewsTopic(topic=topic, fetchSource=True)
    missing_sources.extend(m_s)
missing_sources

In [ ]:
all_sources = pd.read_sql("select * from newsdb.sources", getEngine())
all_sources.shape

In [ ]:
art = pd.DataFrame(articles)

In [ ]:
all_articles = pd.read_sql_table("articles", getEngine())
all_articles.shape

In [ ]:
all_article_contents = pd.read_sql_table("article_contents", getEngine())
all_article_contents.shape

In [ ]:
all_article_contents.keys()

In [ ]:
lenall_article_contents[all_article_contents['article_content']=='']

In [ ]:
len(articles)

In [ ]:
articles[0].article_url

In [ ]:
df = pd.read_sql(query.statement, query.session.bind)

In [ ]:
df[0:3]

In [ ]:
pd.DataFrame(articles)

In [ ]:
for a in articles:
    print(len(a))

In [ ]:
source_name = 'New York Times'
source = sql_utils.getSource(source_name)

In [ ]:
def getNYTSections(api_key):
    url = "https://api.nytimes.com/svc/news/v3/content/section-list.json?api-key=%s" % api_key
    return requests.get(url=url, params={}).json()

In [ ]:
sections_2 = getNYTSections(source.api_key)

In [ ]:
api_key

In [ ]:
for s in sections_2['results']:
    print(s['section'])

In [ ]:
api_key = source.api_key
url = "https://api.nytimes.com/svc/news/v3/content/section-list.json?api-key=%s" % api_key
r = requests.get(url=url, params={})
sections = r.json()

In [ ]:
sections['results']

In [ ]:
with open('../raw_data/nytwire.json') as f:
  data = json.load(f)

In [ ]:
data.keys()

In [ ]:
data['results'][499]

In [ ]:
pipelineNYTHeadlines(fetchSource=True, verbose=Verbose.INFO)

In [ ]:
source_name = 'New York Times'
source = sql_utils.getSource(source_name)

In [ ]:
pipelineNYTNewsWire(fetchSource=True, verbose=Verbose.INFO)

In [ ]:
sections = getNYTSections(source.api_key)

In [ ]:
sections.keys()

In [ ]:
createNlpModel(attributes=['title', 'description'])

In [ ]:
peer_uuids, df = get_peers_from_text('vice-president', 5)
df

In [ ]:
url = "https://rss.nytimes.com/services/xml/rss/nyt/Arts.xml"
d = feedparser.parse(url)
d['feed']

In [ ]:
d.keys()

In [ ]:
d['feed'].keys()

In [ ]:
d['feed']['published']

In [ ]:
d['entries'][0].keys()